In [1]:
from aim2dat.strct import StructureCollection

strct_c = StructureCollection()

The structures can be added to the object via the different `append*` functions of the object:

In [2]:
from ase.spacegroup import crystal
import aiida
from aiida.orm import StructureData
from aim2dat.strct import Structure

structure_dict = {
    "label": "Benzene",
    "elements": ["C"] * 6 + ["H"] * 6,
    "pbc": False,
    "positions": [
        [-0.7040, -1.2194, -0.0000],
        [0.7040, -1.2194, -0.0000],
        [-1.4081, -0.0000, -0.0000],
        [1.4081, 0.0000, 0.0000],
        [-0.7040, 1.2194, 0.0000],
        [0.7040, 1.2194, -0.0000],
        [-1.2152, -2.1048, -0.0000],
        [1.2152, -2.1048, 0.0000],
        [-2.4304, -0.0000, 0.0000],
        [2.4304, 0.0000, 0.0000],
        [-1.2152, 2.1048, -0.0000],
        [1.2152, 2.1048, 0.0000],
    ],
}
strct_c.append(**structure_dict)

structure = Structure(
    elements=["O", "H", "H"],
    positions=[[0.0, 0.0, 0.119], [0.0, 0.763, -0.477], [0.0, -0.763, -0.477]],
    pbc=False,
)
strct_c.append_structure(structure, label="Water")

a = 4.066 * 2.0
GaAs_conv = crystal(
    ("Ga", "As"),
    basis=((0.0, 0.0, 0.0), (0.75, 0.75, 0.75)),
    spacegroup=216,
    cellpar=[a, a, a, 90, 90, 90],
    primitive_cell=False,
)
strct_c.append_from_ase_atoms("GaAs", GaAs_conv)

aiida.load_profile("tests")
unit_cell = [[3.0, 0.0, 0.0], [0.0, 3.0, 0.0], [0.0, 0.0, 3.0]]
structure = StructureData(cell=unit_cell)
structure.label = "Li"
structure.append_atom(position=(0.0, 0.0, 0.0), symbols="Li")
structure.append_atom(position=(1.5, 1.5, 1.5), symbols="Li")
strct_c.append_from_aiida_structuredata(structure)

Alternatively, a list of dictionaries can be passed upon initialization of the object:

In [3]:
strct_c2 = StructureCollection(structures=[structure_dict])

A summary of the object is given by its string representation:

In [4]:
print(strct_c)

----------------------------------------------------------------------
------------------------ Structure Collection ------------------------
----------------------------------------------------------------------

 - Number of structures: 4
 - Elements: As-C-Ga-H-Li-O

                              Structures                              
  - Benzene             C6H6                [False False False]
  - Water               OH2                 [False False False]
  - GaAs                Ga4As4              [True  True  True ]
  - Li                  Li2                 [True  True  True ]
----------------------------------------------------------------------


Additionally, a pandas data frame can be created based on the object's content:

In [5]:
strct_c2.create_pandas_df()

,label,el_conc_C,el_conc_H,nr_atoms,nr_atoms_C,nr_atoms_H
0,Benzene,0.5,0.5,12,6,6


In [6]:
print(strct_c[1])

----------------------------------------------------------------------
-------------------------- Structure: Water --------------------------
----------------------------------------------------------------------

 Formula: OH2
 PBC: [False False False]

                                Sites                                 
  - O  None  [  0.0000   0.0000   0.1190]
  - H  None  [  0.0000   0.7630  -0.4770]
  - H  None  [  0.0000  -0.7630  -0.4770]
----------------------------------------------------------------------


In [7]:
print(strct_c["Water"])

----------------------------------------------------------------------
-------------------------- Structure: Water --------------------------
----------------------------------------------------------------------

 Formula: OH2
 PBC: [False False False]

                                Sites                                 
  - O  None  [  0.0000   0.0000   0.1190]
  - H  None  [  0.0000   0.7630  -0.4770]
  - H  None  [  0.0000  -0.7630  -0.4770]
----------------------------------------------------------------------


In [8]:
print(strct_c.get_structure(3))

----------------------------------------------------------------------
--------------------------- Structure: Li ----------------------------
----------------------------------------------------------------------

 Formula: Li2
 PBC: [True True True]

                                 Cell                                 
 Vectors: - [  3.0000   0.0000   0.0000]
          - [  0.0000   3.0000   0.0000]
          - [  0.0000   0.0000   3.0000]
 Lengths: [  3.0000   3.0000   3.0000]
 Angles: [ 90.0000  90.0000  90.0000]
 Volume: 27.0000

                                Sites                                 
  - Li None  [  0.0000   0.0000   0.0000] [  0.0000   0.0000   0.0000]
  - Li None  [  1.5000   1.5000   1.5000] [  0.5000   0.5000   0.5000]
----------------------------------------------------------------------


In [9]:
del strct_c["Benzene"]
strct_c.labels

['Water', 'GaAs', 'Li']

Two structure collection objects can be merged into one using `+`:

In [10]:
print(strct_c + strct_c2)

----------------------------------------------------------------------
------------------------ Structure Collection ------------------------
----------------------------------------------------------------------

 - Number of structures: 4
 - Elements: As-C-Ga-H-Li-O

                              Structures                              
  - Water               OH2                 [False False False]
  - GaAs                Ga4As4              [True  True  True ]
  - Li                  Li2                 [True  True  True ]
  - Benzene             C6H6                [False False False]
----------------------------------------------------------------------


In [11]:
strct_c.store_in_aiida_db(group_label="test")
strct_c = StructureCollection()
strct_c.import_from_aiida_db(group_label="test")

Storing data as group `test` in the AiiDA database.


In [12]:
strct_c.store_in_hdf5_file("test.h5")
strct_c = StructureCollection()
strct_c.import_from_hdf5_file("test.h5")

In [13]:
from aim2dat.strct import StructureOperations

strct_c += strct_c2
strct_op = StructureOperations(structures=strct_c, verbose=False)

In [14]:
strct_op.supported_output_formats

['dict', 'DataFrame']

In [15]:
print("Analysis methods: ", strct_op.analysis_methods)
print("Manipulation methods: ", strct_op.manipulation_methods)

Analysis methods:  ['determine_point_group', 'determine_space_group', 'calculate_distance', 'calculate_angle', 'calculate_dihedral_angle', 'calculate_voronoi_tessellation', 'calculate_coordination', 'calculate_ffingerprint']
Manipulation methods:  ['delete_atoms', 'scale_unit_cell', 'substitute_elements']


In [16]:
print("One structure: ", strct_op[["Benzene"]].calculate_distance(2, 3))
print("One structure: ", strct_c["Benzene"].calculate_distance(site_index1=2, site_index2=3))

One structure:  {'Benzene': 2.8162}
One structure:  2.8162


In [17]:
print("Multiple structures: ", strct_op[0,1].calculate_distance(0, 1))

Multiple structures: 

 {'Li': 2.598076211353316, 'GaAs': 5.750192323029818}


In [18]:
strct_op.output_format = "DataFrame"
strct_op[0, 1].calculate_distance(0, 1)

,<function calculate_distance at 0x7f73735fac20>
Li,2.598076
GaAs,5.750192


In [19]:
subst_structure = strct_op["GaAs"].substitute_elements(("Ga", "Al"), change_label=True)
print(subst_structure)
print(strct_op.structures.labels)

----------------------------------------------------------------------
--------------------- Structure: GaAs_subst-GaAl ---------------------
----------------------------------------------------------------------

 Formula: Al4As4
 PBC: [True True True]

                                 Cell                                 
 Vectors: - [  8.0987   0.0000   0.0000]
          - [  0.0000   8.0987   0.0000]
          - [  0.0000   0.0000   8.0987]
 Lengths: [  8.0987   8.0987   8.0987]
 Angles: [ 90.0000  90.0000  90.0000]
 Volume: 531.1797

                                Sites                                 
  - Al None  [  0.0000   0.0000   0.0000] [  0.0000   0.0000   0.0000]
  - Al None  [  0.0000   4.0493   4.0493] [  0.0000   0.5000   0.5000]
  - Al None  [  4.0493   0.0000   4.0493] [  0.5000   0.0000   0.5000]
  - Al None  [  4.0493   4.0493   0.0000] [  0.5000   0.5000   0.0000]
  - As None  [  6.0740   6.0740   6.0740] [  0.7500   0.7500   0.7500]
  - As None  [  2.0247   2.02

In [20]:
subst_structure = strct_op["GaAs"].substitute_elements(("Ga", "Al"), change_label=False)
print(subst_structure)
print(strct_op.structures.labels)
print(strct_op.structures["GaAs"])

----------------------------------------------------------------------
-------------------------- Structure: GaAs ---------------------------
----------------------------------------------------------------------

 Formula: Al4As4
 PBC: [True True True]

                                 Cell                                 
 Vectors: - [  8.0987   0.0000   0.0000]
          - [  0.0000   8.0987   0.0000]
          - [  0.0000   0.0000   8.0987]
 Lengths: [  8.0987   8.0987   8.0987]
 Angles: [ 90.0000  90.0000  90.0000]
 Volume: 531.1797

                                Sites                                 
  - Al None  [  0.0000   0.0000   0.0000] [  0.0000   0.0000   0.0000]
  - Al None  [  0.0000   4.0493   4.0493] [  0.0000   0.5000   0.5000]
  - Al None  [  4.0493   0.0000   4.0493] [  0.5000   0.0000   0.5000]
  - Al None  [  4.0493   4.0493   0.0000] [  0.5000   0.5000   0.0000]
  - As None  [  6.0740   6.0740   6.0740] [  0.7500   0.7500   0.7500]
  - As None  [  2.0247   2.02

In [21]:
subst_structures = strct_op[strct_op.structures.labels].substitute_elements(
    ("Al", "Ga"), change_label=False
)
print(subst_structures)

----------------------------------------------------------------------
------------------------ Structure Collection ------------------------
----------------------------------------------------------------------

 - Number of structures: 4
 - Elements: As-C-Ga-H-Li-O

                              Structures                              
  - Li                  Li2                 [True  True  True ]
  - GaAs                Ga4As4              [True  True  True ]
  - Water               OH2                 [False False False]
  - Benzene             C6H6                [False False False]
----------------------------------------------------------------------


In [22]:
from aim2dat.strct.ext_analysis import calculate_prdf

output = strct_op["Benzene"].perform_analysis(calculate_prdf, {"r_max": 7.5})